## 1. データをクローリングする

In [ ]:
! mkdir -p html excel csv json tokenized vector log

In [ ]:
# 事故事例(Excel系)と、事例・状況・原因・対策（HTML系）の2種類を取得する。
! ./get_doc.sh > ./log/get_doc.log 2>&1 

## 2. 取得したデータをCSVまたはJSONに変換する。
* 事故事例のExcelをCSVに変換する。
* 事例・状況・原因・対策のHTMLはJSONに変換する。

In [ ]:
# Excel to CSV
!python excel_to_csv.py ./excel/ ./csv/ > ./log/excel_to_csv.log 2>&1

In [55]:
# HTML to JSON
# !python html_to_json.py ./html/ ./json/ > ./log/html_to_json.log 2>&1 
!python html_to_csv.py ./html/ ./csv/anzen_all.csv > ./log/html_to_csv.log 2>&1 

## 3. ElasticsearchにIndexを登録する

In [60]:
# 事例・状況・原因・対策用のインデックスを登録
#! curl -XPUT -H "Content-Type: application/json" elasticsearch:9200/anzen -d @es_anzen_schema.txt
! curl -XPUT -H "Content-Type: application/json" elasticsearch:9200/anzen -d @es_anzen_csv_schema.txt

{"acknowledged":true,"shards_acknowledged":true,"index":"anzen"}

In [ ]:
# 事例用のインデックスを登録
! curl -XPUT -H "Content-Type: application/json" elasticsearch:9200/accident -d @es_accident_schema.txt

## 4. Elasticsearchにデータをロードする

In [61]:
# 事例・状況・原因・対策データをロード(anzenインデックス)
#! ./load_anzen_bulk_es.sh > ./log/load_anzen_bulk_es.log 2>&1
! python load_anzen_es.py --host elasticsearch --index anzen --input_csv ./csv/anzen_all.csv  > ./log/load_anzen_es.log 2>&1

In [ ]:
# 事例用データをロード(accidentインデックス)
!python load_accident_es.py --host elasticsearch --index accident --input_dir ./csv/  > ./log/load_accident_es.log 2>&1

## 5. Elasticsearchでトークナイズし、結果をCSV形式とテキスト形式で保存する

In [62]:
# 事例・状況・原因・対策データの文章をトークナイズし、CSVとTXTファイルとして保存する
!python es_anzen_tokenize.py --host elasticsearch --index anzen --output tokenized/anzen_tokenized > ./log/es_anzen_tokenize.log 2>&1

In [ ]:
# 事例用データの文章をトークナイズし、CSVとTXTファイルとして保存する
!python es_accident_tokenize.py --host elasticsearch --index accident --output tokenized/accident_tokenized

## 6. GloVeで単語を学習する

In [ ]:
# GloVeを取得する。最初に1回だけ実行すれば良い。
! git clone https://github.com/stanfordnlp/GloVe.git
! cd GloVe && make

In [8]:
# トークナイズしたanzenとaccidentのテキストファイルを結合する。
! cat tokenized/anzen_tokenized.txt tokenized/accident_tokenized.txt > tokenized/all_tokens.txt

In [ ]:
# GloVeで学習する。
! ./glove.sh 

# SCDVで文章をベクトル化する

In [63]:
# anzenとaccidentのCSVをマージする。ベクトル化に必要な列のみ選別し、列名を統一する。
!python merge_csv.py --input_anzen tokenized/anzen_tokenized.csv --input_accident tokenized/accident_tokenized.csv --output_csv tokenized/merge_tokenized.csv

In [64]:
!python scdv.py --csv_file ./tokenized/merge_tokenized.csv 

Clustering Done...
Cluster Assignments Saved...
Probabilities of Cluster Assignments saved...
100%|████████████████████████████████████| 5893/5893 [00:00<00:00, 18155.08it/s]
train size:53033  vector size:1000
test size:5893  vector size:1000
Test start...
Report
              precision    recall  f1-score   support

       cause   0.565625  0.413242  0.477573       438
    measures   0.695560  0.658000  0.676259       500
   situation   0.660000  0.474333  0.551971       487
       title   0.351852  0.218391  0.269504        87
      その他の事業   0.488636  0.176230  0.259036       244
      保健・衛生業   0.000000  0.000000  0.000000         3
        保健衛生   0.000000  0.000000  0.000000         1
       保健衛生業   0.014286  0.043478  0.021505        23
          商業   0.511029  0.344913  0.411852       403
         官公署   0.033333  0.500000  0.062500         2
         建設業   0.657814  0.782979  0.714960      1645
       接客娯楽業   0.280000  0.102941  0.150538        68
      教育・研究業   0.000000  0.000000

In [65]:
# 全データについてSCDVベクトルを算出し、Elasticsearchに登録する
!python scdv_to_es.py --host elasticsearch --input_csv ./tokenized/merge_tokenized.csv > ./log/scdv_to_es.log 2>&1 